In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np

In [3]:
url = 'https://www.daraz.pk/'
driver.get(url)

In [4]:
search_bar = driver.find_element(By.ID, 'q')
search_bar.send_keys('Mobiles' + Keys.RETURN)
time.sleep(5)

In [104]:
#time.sleep(5)
next_page_button = driver.find_element(By.CLASS_NAME, 'ant-pagination-next')
next_page_button.click()

In [7]:
from selenium.common.exceptions import TimeoutException

def scrape_reviews(url, max_pages=5):
    driver.get(url)
    time.sleep(3)
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'module_product_review')))
    
    page_number = 1
    all_reviews = []

    while page_number <= max_pages:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) 

        reviews = driver.find_elements(By.CLASS_NAME, 'review-content-sl')
        for review in reviews:
            all_reviews.append(review.text)

        driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center', behavior: 'smooth'});", driver.find_element(By.XPATH, '//*[@id="module_product_detail"]/h2'))

        try:
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ant-pagination-next')))
        except TimeoutException:
            break

        if "disabled" in next_button.get_attribute("class"):
            break

        # driver.execute_script("window.scrollTo(1400, 1250);")
        time.sleep(4)
        
        next_button.click()
        page_number += 1

    if not all_reviews:
        return "None"
    else:
        return all_reviews


In [8]:
def scrape_phone_info(phone_url, unique_id):
    driver.get(phone_url)
    time.sleep(3)
    check=driver.find_element(By.XPATH,'//*[@id="J_breadcrumb"]/li[2]/span/a/span').text.split('(')[0].strip()
    if check == 'Like New Phones' or check == 'Mobiles' or check=='Feature Phones':
        driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center', behavior: 'smooth'});", driver.find_element(By.ID, 'module_product_detail'))
        time.sleep(3)

        try:
            product_title = driver.find_element(By.ID, 'module_product_title_1').text.split('(')[0].strip()
        except:
            product_title = None

        try:
            price = driver.find_element(By.XPATH, '//*[@id="module_product_price_1"]/div/div/span').text.split('(')[0].strip()
        except:
            price = None

        try:
            rating = driver.find_element(By.CLASS_NAME, 'score').text
        except:
            rating = None
        
        try:
            specifications=driver.find_elements(By.XPATH, '//*[@id="module_product_detail"]/div/div[1]/div')
            spec=[]
            for i in specifications:
                spec.append(i.text)
        except:
            specifications=None

        try:
            free_shipping_status = driver.find_element(By.XPATH, '//*[@id="module_seller_delivery"]/div/div/div[3]/div/div[1]/div/div[1]/div[1]/div[1]/span[1]').text.strip()
        except:
            free_shipping_status = None

        try:
            seller_rating = driver.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[1]/div[2]').text
        except:
            seller_rating = None

        try:
            ship_on_time = driver.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[2]/div[2]').text
        except:
            ship_on_time = None

        reviews_url = phone_url
        reviews = scrape_reviews(reviews_url)

        driver.back()

        return {
            'ID': unique_id,
            'Product Title': product_title,
            'Price': price,
            'Rating': rating,
            'Free Shipping Status': free_shipping_status,
            'Seller Rating': seller_rating,
            'Ship on Time': ship_on_time,
            'Product Link': phone_url,
            'Reviews': reviews,
            'Specifications':spec
        }
    else:
        driver.back()

def click_next_page():
    try:
        next_page_button = driver.find_element(By.CLASS_NAME, 'ant-pagination-next')
        next_page_button.click()
        time.sleep(5) 
        return True
    except:
        return False


id_counter = 1
phone_data_list = []
count=0

for page in range(1, 6):
    print(f"Scraping data from page {page}")

    phone_links = driver.find_elements(By.ID, 'id-a-link')

    phone_links_hrefs = [link.get_attribute('href') for link in phone_links]
    print(f"Number of phone links on page {page}: {len(phone_links_hrefs)}")

    visited_links = set()

    for href in phone_links_hrefs:
        if href not in visited_links:
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            driver.get(href) 
            phone_info = scrape_phone_info(driver.current_url, f'{id_counter:04d}')

            if phone_info:
                phone_data_list.append(phone_info)
                id_counter += 1

            visited_links.add(href) 
            count+=1
            
            if count==40:
                click_next_page()
                count=0

    print(f"Number of phone data collected on page {page}: {len(phone_data_list)}")

    if not click_next_page():
        print("No more pages available")
        break

Scraping data from page 1
Number of phone links on page 1: 80
Number of phone data collected on page 1: 40
Scraping data from page 2
Number of phone links on page 2: 80
Number of phone data collected on page 2: 80
Scraping data from page 3
Number of phone links on page 3: 80
Number of phone data collected on page 3: 120
Scraping data from page 4
Number of phone links on page 4: 80
Number of phone data collected on page 4: 160


In [9]:
phone_df = pd.DataFrame(phone_data_list)
#display(phone_df)

reviews= phone_df[['ID', 'Reviews']].copy()
phone_info = phone_df.drop(['Reviews'], axis=1).copy()

reviews.to_csv('reviews.csv', index=False)
phone_info.to_csv('phone_info.csv', index=False)

In [10]:
#loading the reviews and phone data for normalization, so it can be processed by chatbot
reviews = pd.read_csv('reviews.csv')
phone_df = pd.read_csv('phone_info.csv')


phone_df

,ID,Product Title,Price,Rating,Free Shipping Status,Seller Rating,Ship on Time,Product Link,Specifications
0,1,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,"Rs. 34,999",4.4,Free Delivery,New Seller,100%,https://www.daraz.pk/products/samsung-a14-4gb6...,"[""• Samsung A14 with 4GB/64GB memory.• 5000 mA..."
1,2,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,"Rs. 49,999",4.7,Standard Delivery,92%,100%,https://www.daraz.pk/products/redmi-note-12-8-...,['Redmi Note 12 8GB RAM + 128GB ROM PTA Approv...
2,3,Like New Phones - Used Apple iPhone X - Silver...,"Rs. 106,999",5.0,Fastest Delivery,92%,100%,https://www.daraz.pk/products/like-new-phones-...,"[""Model: Apple\nModel: iPhone X\nBattery Healt..."
3,4,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - An...,"Rs. 19,399",4.7,Standard Delivery,92%,100%,https://www.daraz.pk/products/redmi-a2-3gb64gb...,['Display: 6.52” HD+ Dot Drop displayResolutio...
4,5,Infinix Note 30 Pro 8-256 GB PTA Approved With...,"Rs. 64,499",4.7,Free Delivery,New Seller,100%,https://www.daraz.pk/products/infinix-note-30-...,['Release Date2023-05-22SIM SupportDual SIM (N...
...,...,...,...,...,...,...,...,...,...
155,156,Itel Super Guru 600 - Ultra Slim Design - Dual...,"Rs. 4,599",NaN,Free Delivery,87%,100%,https://www.daraz.pk/products/600-28-pta-1900m...,['pre { white-space: pre-wrap; }• Ultra slim d...
156,157,Vgotel New 12 - 2GB RAM 64GB ROM - 6.52 Inches...,"Rs. 16,999",5.0,Free Delivery,88%,100%,https://www.daraz.pk/products/12-2gb-64gb-652-...,['PTA Approved\n6.52 Inches Display\n2GB RAM\n...
157,158,Sparx S7 2GB Ram 32GB Rom Tripple Camera 4000m...,"Rs. 14,999",3.6,Free Delivery,92%,96%,https://www.daraz.pk/products/sparx-s7-2gb-ram...,['• The Sparx S7 boasts 2GB RAM and 32GB ROM f...
158,159,Infinix Note 30 RAM 8 GB ROM 256 GB Front Came...,"Rs. 46,999",4.8,Free Delivery,95%,100%,https://www.daraz.pk/products/30-8-gb-256-gb-1...,['BuildOSAndroid 13 OS UIXOS Dimensions168.6 x...


In [11]:
import numpy as np
import re

phone_df['Seller Rating'] = phone_df['Seller Rating'].replace('New Seller', np.nan)
phone_df['Seller Rating'] = pd.to_numeric(phone_df['Seller Rating'].replace('not enough data', np.nan).str.rstrip('%'), errors='coerce')

phone_df['Price'] = phone_df['Price'].apply(lambda x: re.findall(r'\d+', str(x)))
phone_df['Price'] = phone_df['Price'].apply(lambda x: int(''.join(x)) if x else np.nan)

def convert_price(price):
    try:
        return int(''.join(re.findall(r'\d+', str(price))))
    except ValueError:
        return np.nan

phone_df['Price'] = phone_df['Price'].apply(convert_price)
phone_df['Rating'] = pd.to_numeric(phone_df['Rating'], errors='coerce')
phone_df['Ship on Time'] = pd.to_numeric(phone_df['Ship on Time'].str.rstrip('%'), errors='coerce')

phone_df = phone_df.fillna(0)

display(phone_df)


,ID,Product Title,Price,Rating,Free Shipping Status,Seller Rating,Ship on Time,Product Link,Specifications
0,1,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,34999,4.4,Free Delivery,0.0,100.0,https://www.daraz.pk/products/samsung-a14-4gb6...,"[""• Samsung A14 with 4GB/64GB memory.• 5000 mA..."
1,2,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,49999,4.7,Standard Delivery,92.0,100.0,https://www.daraz.pk/products/redmi-note-12-8-...,['Redmi Note 12 8GB RAM + 128GB ROM PTA Approv...
2,3,Like New Phones - Used Apple iPhone X - Silver...,106999,5.0,Fastest Delivery,92.0,100.0,https://www.daraz.pk/products/like-new-phones-...,"[""Model: Apple\nModel: iPhone X\nBattery Healt..."
3,4,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - An...,19399,4.7,Standard Delivery,92.0,100.0,https://www.daraz.pk/products/redmi-a2-3gb64gb...,['Display: 6.52” HD+ Dot Drop displayResolutio...
4,5,Infinix Note 30 Pro 8-256 GB PTA Approved With...,64499,4.7,Free Delivery,0.0,100.0,https://www.daraz.pk/products/infinix-note-30-...,['Release Date2023-05-22SIM SupportDual SIM (N...
...,...,...,...,...,...,...,...,...,...
155,156,Itel Super Guru 600 - Ultra Slim Design - Dual...,4599,0.0,Free Delivery,87.0,100.0,https://www.daraz.pk/products/600-28-pta-1900m...,['pre { white-space: pre-wrap; }• Ultra slim d...
156,157,Vgotel New 12 - 2GB RAM 64GB ROM - 6.52 Inches...,16999,5.0,Free Delivery,88.0,100.0,https://www.daraz.pk/products/12-2gb-64gb-652-...,['PTA Approved\n6.52 Inches Display\n2GB RAM\n...
157,158,Sparx S7 2GB Ram 32GB Rom Tripple Camera 4000m...,14999,3.6,Free Delivery,92.0,96.0,https://www.daraz.pk/products/sparx-s7-2gb-ram...,['• The Sparx S7 boasts 2GB RAM and 32GB ROM f...
158,159,Infinix Note 30 RAM 8 GB ROM 256 GB Front Came...,46999,4.8,Free Delivery,95.0,100.0,https://www.daraz.pk/products/30-8-gb-256-gb-1...,['BuildOSAndroid 13 OS UIXOS Dimensions168.6 x...


In [12]:
#Normalizing Revews and Sentiment Analysis

In [13]:
#Normalizing
import re
reviews = reviews.dropna(subset=['Reviews'])
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text


In [14]:
#Sentiment Analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def analyze_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    
    if sentiment_scores['compound'] >= 0.05:
        return 'Positive'
    elif sentiment_scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [15]:
reviews['Reviews'] = reviews['Reviews'].apply(normalize_text)
reviews['Review Analysis'] = reviews['Reviews'].apply(analyze_sentiment)

display(reviews)

,ID,Reviews,Review Analysis
0,1,genuine samsung mobile well packed pta approve...,Positive
1,2,this is the initial review the phone is the sa...,Positive
2,3,received as promised almost 1010 condition gen...,Positive
3,4,alhamdulillah same recieved as shown in pictur...,Positive
4,5,acha mobile hai and same cheez delivered ki ha...,Positive
...,...,...,...
154,155,well and protected packed delivered well befor...,Positive
156,157,befor 1111 sale excellent masha allah and jaza...,Positive
157,158,mashallah this seller rocks once again shayed ...,Negative
158,159,i received my product before delievery date th...,Positive


In [16]:
#saving new reviews with sentiment and new phone_info
reviews.to_csv('new_reviews.csv', index=False)
phone_df.to_csv('new_phone.csv', index=False)


In [92]:
new_review= pd.read_csv('new_reviews.csv')
new_phone= pd.read_csv('new_phone.csv')

In [93]:
database = pd.merge(new_review, new_phone, on='ID', how='inner')
database.to_csv('database.csv', index=False)

In [111]:
#chatbot implementation

import pandas as pd
import re
import spacy

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

def process_query(user_query):
    query_doc = nlp(user_query)
    intents = extract_intents(query_doc)

    response = ""
    for intent in intents:
        if intent == 'budget':
            budget = extract_budget(query_doc)
            relevant_phones = get_phones_within_budget(new_phone, budget)
            sorted_phones = sort_phones(relevant_phones)
            response += generate_response(sorted_phones)

        elif intent == 'brand':
            brand = extract_brand(query_doc)
            relevant_phones = get_phones_by_brand(new_phone, brand)
            sorted_phones = sort_phones(relevant_phones)
            response += generate_response(sorted_phones)

        elif intent == 'budget_between':
            budget_range = extract_budget_range(query_doc)
            if budget_range is not None:
                relevant_phones = get_phones_within_budget_range(new_phone, budget_range)
                sorted_phones = sort_phones(relevant_phones)
                response += generate_response(sorted_phones)

        elif intent == 'feature':
            feature = extract_feature(query_doc)
            relevant_phones = get_phones_with_feature(new_phone, feature)
            sorted_phones = sort_phones(relevant_phones)
            response += generate_response(sorted_phones)

        elif intent == 'statistics':
            statistics_response = generate_statistics_response(database)
            response += generate_statistics_prompt(statistics_response)

        elif intent == 'links':
            statistics_response = generate_statistics_response(database)
            response += generate_links_prompt(statistics_response)

        elif intent == 'review':
            review_sentiment = extract_review_sentiment(query_doc)
            relevant_reviews = get_reviews_by_sentiment(new_review, review_sentiment)
            sorted_reviews = sort_reviews(relevant_reviews)
            response += generate_review_response(sorted_reviews)
        
        elif intent == 'rating':
            rating = extract_float_from_string(user_query)
            relevant_phones = sort_rating(new_phone, rating)
            response += generate_rating_response(relevant_phones,rating)

        elif intent == 'greeting':
            response += generate_greeting_response()
            
        elif intent == 'phone_name':
            phone_name = extract_phone_name(query_doc)
            response += generate_response_by_phone_name(phone_name, new_phone)
            

        else:
            response += generate_general_response()

    return response

def extract_intents(query_text):
    query_doc = nlp(query_text)
    intents = []
    for token in query_doc:
        if token.lower_ in ['budget', 'price', 'affordable', 'range']:
            intents.append('budget')
        elif token.lower_ in ['brand', 'manufacturer', 'model']:
            intents.append('brand')
        elif token.lower_ in ['budget between', '-',]:
            intents.append('budget_between')
        elif token.lower_ in ['feature', 'specification', 'type']:
            intents.append('feature')
        elif token.lower_ in ['review', 'opinion', 'good', 'bad']:
            intents.append('review')
        elif token.lower_ in ['rating', 'rate', 'best']:
            intents.append('rating')
        elif token.lower_ in ['statistics', 'stats', 'data', 'numbers']:
            intents.append('statistics')
        elif token.lower_ in ['links', 'html', 'product links', 'http']:
            intents.append('links')
        elif token.lower_ in ['phone name', 'name']:
            intents.append('phone_name')
        elif token.lower_ in ['hello', 'hi', 'konichiwa']:
            intents.append('greeting')

    if not intents:
        intents.append('general')

    return intents

def extract_numbers_from_string(input_string):
    match = re.search(r'\b\d+\b', input_string)
    
    if match:
        return int(match.group())
    else:
        return None
    
def extract_float_from_string(input_string):
    match = re.search(r'\b\d+\b', input_string)
    
    if match:
        return float(match.group())
    else:
        return None

def extract_budget(query_text):
    query_text = query_text.text
    if any(keyword in query_text.lower() for keyword in ['budget', 'price', 'affordable']):
        query_doc = nlp(query_text)
        for token in query_doc:
            if token.ent_type_ == 'MONEY':
                try:
                    return float(token.text.replace('Rs', '').replace(',', ''))
                except ValueError:
                    continue
        
        numbers_in_query = extract_numbers_from_string(query_text)
        if numbers_in_query is not None:
            return float(numbers_in_query)
    
    return None


def extract_brand(query_text):
    query_text = query_text.text
    query_doc = nlp(query_text)
    for token in query_doc:
        if token.pos_ == 'PROPN':
            return token.text
    return None

def extract_feature(query_text):
    query_text = query_text.text
    query_doc = nlp(query_text)
    for token in query_doc:
        if token.lower_ in ['camera', 'battery', 'display']:
            return token.text
    return None

def extract_review_sentiment(query_text):
    query_text = query_text.text
    query_doc = nlp(query_text)
    for token in query_doc:
        if token.lower_ in ['positive', 'good', 'excellent']:
            return 'Positive'
        elif token.lower_ in ['negative', 'bad', 'poor']:
            return 'Negative'
    return 'Neutral'

def get_phones_within_budget(new_phone, budget):
    return new_phone[new_phone['Price'] <= budget]

def get_phones_by_brand(new_phone, brand):
    return new_phone[new_phone['Product Title'].str.contains(brand, case=False, na=False)]

def get_phones_with_feature(new_phone, feature):
    return new_phone[new_phone['Specifications'].str.contains(feature, case=False, na=False)]

def get_reviews_by_sentiment(new_review, sentiment):
    return new_review[new_review['Review Analysis'] == sentiment]

def sort_phones(relevant_phones):
    return relevant_phones.sort_values(by='Rating', ascending=False)

def generate_response(sorted_phones):
    if not sorted_phones.empty:
        response = "Here are some phones matching your criteria:\n"
        for _, phone in sorted_phones.iterrows():
            response += f"Title: {phone['Product Title']}, Price: {phone['Price']}, Rating: {phone['Rating']}\n"
    else:
        response = "No phones found matching your criteria."

    return response

def generate_review_response(relevant_reviews):
    if not relevant_reviews.empty:
        response = "Here are some reviews matching your criteria:\n"
        for _, review in relevant_reviews.iterrows():
            response += f"Review: {review['Reviews']}, Sentiment: {review['Review Analysis']}\n"
    else:
        response = "No reviews found matching your criteria."

    return response

def generate_greeting_response():
    response="Hello, this is a simple chatbot trained on Phones scrapped from Daraz. Please make use of it!"
    return response

def generate_general_response():
    response="I cannot answer what I do not know. Please make use of specific keywords or you'll get this message"
    return response

def generate_statistics_response(database):
    total_listings = len(database)
    average_price = database['Price'].mean()
    average_rating = database['Rating'].mean()
    database['Review_Count'] = database['Reviews'].apply(lambda x: len(x))
    average_review_count = database['Review_Count'].mean()
    top_5_highest_ratings = database.nlargest(5, 'Rating')
    top_5_most_reviews = database.nlargest(5, 'Review_Count')

    return {
        'total_listings': total_listings,
        'average_price': average_price,
        'average_rating': average_rating,
        'average_review_count': average_review_count,
        'top_5_highest_ratings': top_5_highest_ratings,
        'top_5_most_reviews': top_5_most_reviews
    }

def generate_statistics_prompt(statistics_response):
    total_listings = statistics_response['total_listings']
    average_price = statistics_response['average_price']
    average_rating = statistics_response['average_rating']
    average_review_count = statistics_response['average_review_count']
    top_5_highest_ratings = statistics_response['top_5_highest_ratings']
    top_5_most_reviews = statistics_response['top_5_most_reviews']

    response = (
        f"Statistics:\n"
        f"Total Number of Listings: {total_listings}\n"
        f"Average Product Price: {average_price:.2f}\n"
        f"Average Ratings of Products: {average_rating:.2f}\n"
        f"Average Review Count per Product: {average_review_count:.2f}\n"
        f"\nTop 5 Highest Ratings:\n{top_5_highest_ratings[['Product Title', 'Rating']].to_string(index=False)}\n"
        f"\nTop 5 Most Reviews:\n{top_5_most_reviews[['Product Title', 'Review_Count']].to_string(index=False)}"
    )

    return response

def generate_links_prompt(statistics_response):
    top_5_highest_ratings = statistics_response['top_5_highest_ratings']
    top_5_links = top_5_highest_ratings['Product Link'].tolist()

    response = "Here are the links to the top 5 highest-rated products:\n"
    for link in top_5_links:
        response += f"{link}\n"

    return response

def sort_rating(relevant_phones, rating):
    relevant_phones = relevant_phones[relevant_phones['Rating'] >= rating]
    sorted_phones = relevant_phones.sort_values(by='Rating', ascending=False)
    return sorted_phones

def generate_rating_response(sorted_phones):
    columns_to_display = ['Product Title', 'Price', 'Rating']
    
    if not sorted_phones.empty:
        response = f"Here are some phones with a rating greater than or equal to {rating}:\n"
        for _, phone in sorted_phones.iterrows():
            response += f"Title: {phone['Product Title']}, Price: {phone['Price']}, Rating: {phone['Rating']}\n"
        response += f"\nColumns Displayed: {', '.join(columns_to_display)}"
    else:
        response = f"No phones found with a rating greater than or equal to {rating}."

    return response

def generate_response_by_phone_name(phone_name, new_phone):
    matching_phones = new_phone[new_phone['Product Title'].str.contains(phone_name, case=False, na=False)]

    if not matching_phones.empty:
        response = f"Here are the phones with the name '{phone_name}':\n"
        for _, phone in matching_phones.iterrows():
            response += f"Title: {phone['Product Title']}, Price: {phone['Price']}, Rating: {phone['Rating']}\n"
    else:
        response = f"No phones found with the name '{phone_name}'."

    return response

def extract_phone_name(query_text):
    query_doc = nlp(query_text)
    for ent in query_doc.ents:
        if ent.label_ == 'PRODUCT':
            return ent.text
    for token in query_doc:
        if token.pos_ == 'PROPN':
            return token.text
    return None

def extract_budget_range(query_text):
    query_text = query_text.text
    query_doc = nlp(query_text)
    tokens = [token.text.lower() for token in query_doc]

    if 'between' in tokens and 'and' in tokens:
        index_between = tokens.index('between')
        index_and = tokens.index('and')

        if index_between < index_and - 1 and index_and < len(tokens) - 1:
            lower_bound = extract_numbers_from_string(tokens[index_between + 1])
            upper_bound = extract_numbers_from_string(tokens[index_and + 1])
            
            if lower_bound is not None and upper_bound is not None:
                return lower_bound, upper_bound

    return None

def get_phones_within_budget_range(new_phone, budget_range):
    lower_bound, upper_bound = budget_range
    return new_phone[(new_phone['Price'] >= lower_bound) & (new_phone['Price'] <= upper_bound)]

In [112]:
#panel
import panel as pn

def collect_messages(event):
    user_query = inp.value
    response = process_query(user_query)
    conversation.append(f"User: {user_query}")
    conversation.append(f"Chatbot: {response}")
    conversation_panel.object = "\n".join(conversation)

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!", button_type='primary', width=80, margin=(0, 10, 0, 0))
button_conversation.on_click(collect_messages)

conversation = ["Chatbot: Hello! Looking for some new phones on daraz?"]
conversation_panel = pn.pane.Markdown("", width=500, height=300)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.Row(conversation_panel),
)

dashboard.show()

Launching server at http://localhost:14606
